In [1]:
import collections
import json
import os

from absl import app
from absl import flags

import numpy as np
import pandas as pd
import tensorflow as tf
import tqdm

2024-07-11 13:52:25.840847: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-11 13:52:26.215590: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-11 13:52:26.218776: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-11 13:52:27.508810: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
input_path = "/home/pbr-student/personal/thesis/jrdb/train_dataset_with_activity" #/labels/labels_2d_pose_stitched_coco/"
scene = "bytes-cafe-2019-02-07_0"

In [10]:
# from human scene transformer, data subfolder
def get_file_handle(path, mode='rt'):
  file_handle = open(path, mode)
  return file_handle
scene_data_file = get_file_handle(
      os.path.join(input_path, 'labels', 'labels_3d', scene + '.json')
  )

# get agents_dict_from_detections
scene_data = json.load(scene_data_file)
agents = collections.defaultdict(list)
for frame in scene_data['labels']:
    ts = int(frame.split('.')[0])
    for det in scene_data['labels'][frame]:
      agents[det['label_id']].append((ts, det))
agents_dict = agents

In [15]:
# get_agents_features_with_box
max_distance_to_robot = 10
agents_pos_dict = collections.defaultdict(dict)
for agent_id, agent_data in agents_dict.items():
    for (ts, agent_instance) in agent_data:
        if agent_instance['attributes']['distance'] <= max_distance_to_robot:
            agents_pos_dict[(ts, agent_id)] = {
                'p': np.array([agent_instance['box']['cx'],
                                agent_instance['box']['cy'],
                                agent_instance['box']['cz']]),
                # rotation angle is relative to negatiev x axis of robot
                'yaw': np.pi - agent_instance['box']['rot_z'],
                'l': agent_instance['box']['l'],
                'w': agent_instance['box']['w'],
                'h': agent_instance['box']['h']
            }
agents_features = agents_pos_dict

In [16]:
agents_df = pd.DataFrame.from_dict(
        agents_features, orient='index'
    ).rename_axis(['timestep', 'id']) 

In [20]:
# get agents keypoints
keypoint_path = os.path.join(input_path, 'labels', 'labels_2d_pose_coco')
scene_data_file = get_file_handle(os.path.join(keypoint_path, scene + '_image0.json'))
scene_data = json.load(scene_data_file)

agents_keypoints = collections.defaultdict(dict)

for frame in scene_data['annotations']:
    print(frame)
    ts = int(frame.split('.')[0])
    for det in scene_data['labels'][frame]:
        agents_keypoints[(ts, det['label_id'])] = {
            'keypoints': np.array(det['keypoints']).reshape(33, 3)}

{'id': 2764, 'track_id': 8, 'image_id': 542, 'category_id': 1, 'is_crowd': 0, 'keypoints': [563.57, -178.91, 0, 587.46, -136.51, 0, 561.96, -134.31, 0, 568.35, -1.95, 0, 608.45, -29.85, 0, 648.95, -9.85, 0, 558.85, 133.25, 1, 715.95, 128.7, 1, 639.5, 234.4, 1, 463.85, 198.15, 2, 636.48, 268.25, 1, 644.3, 269.27, 1, 618.6, 268.95, 2, 661.1, 468.55, 0, 525.05, 476.1, 2, 703.6, 636.2, 0, 506.5, 660.25, 0], 'num_keypoints': 17}


AttributeError: 'dict' object has no attribute 'split'